# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df["Max Temp"]=city_data_df["Max Temp"]-273.15
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,meulaboh,4.1363,96.1285,23.88,88,97,1.61,ID,25200
1,1,lebu,-37.6167,-73.6500,18.32,81,54,3.99,CL,-10800
2,2,carnarvon,-24.8667,113.6333,25.69,71,1,1.96,AU,28800
3,3,kavaratti,10.5669,72.6420,27.34,67,81,2.41,IN,19800
4,4,lata,40.1629,-8.3327,6.71,72,0,1.71,PT,0


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size="Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600    
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
# Specifications:
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

cleaned_df=city_data_df[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>=21) & (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"]==0) ]
cleaned_df

# Drop any rows with null values
cleaned_df=cleaned_df.dropna(how="any")

# Display sample data
cleaned_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,bhatkal,13.9667,74.5667,25.62,60,0,1.59,IN,19800
62,62,pisco,-13.7000,-76.2167,24.03,83,0,4.12,PE,-18000
73,73,miraflores,19.2167,-98.8083,23.32,26,0,1.70,MX,-21600
116,116,port hueneme,34.1478,-119.1951,24.33,45,0,2.57,US,-28800
125,125,bredasdorp,-34.5322,20.0403,21.90,78,0,2.89,ZA,7200
153,153,katsina,12.9894,7.6006,26.94,28,0,2.06,NG,3600
226,226,cabo san lucas,22.8909,-109.9124,26.61,46,0,3.60,MX,-25200
276,276,lompoc,34.6391,-120.4579,21.55,69,0,2.57,US,-28800
360,360,saint-pierre,-21.3393,55.4781,25.82,83,0,2.06,RE,14400
473,473,kiama,-34.6833,150.8667,21.11,64,0,3.13,AU,39600


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=cleaned_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
36,bhatkal,IN,13.9667,74.5667,60,
62,pisco,PE,-13.7000,-76.2167,83,
73,miraflores,MX,19.2167,-98.8083,26,
116,port hueneme,US,34.1478,-119.1951,45,
125,bredasdorp,ZA,-34.5322,20.0403,78,
153,katsina,NG,12.9894,7.6006,28,
226,cabo san lucas,MX,22.8909,-109.9124,46,
276,lompoc,US,34.6391,-120.4579,69,
360,saint-pierre,RE,-21.3393,55.4781,83,
473,kiama,AU,-34.6833,150.8667,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df["Lat"][index]
    longitude=hotel_df["Lng"][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bhatkal - nearest hotel: Sree Mahasati Store And Tea Hotel
pisco - nearest hotel: La Portada
miraflores - nearest hotel: Hotel Hacienda Panoaya
port hueneme - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
katsina - nearest hotel: Fadama view Hotel
cabo san lucas - nearest hotel: Comfort Rooms
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
saint-pierre - nearest hotel: Tropic Hotel
kiama - nearest hotel: Nova Kiama
morondava - nearest hotel: Hotel Sharon Menabe
clarence town - nearest hotel: Erringhi Hotel
la palma - nearest hotel: La Quinta Inn & Suites by Wyndham Buena Park


,City,Country,Lat,Lng,Humidity,Hotel Name
36,bhatkal,IN,13.9667,74.5667,60,Sree Mahasati Store And Tea Hotel
62,pisco,PE,-13.7000,-76.2167,83,La Portada
73,miraflores,MX,19.2167,-98.8083,26,Hotel Hacienda Panoaya
116,port hueneme,US,34.1478,-119.1951,45,No hotel found
125,bredasdorp,ZA,-34.5322,20.0403,78,Victoria Hotel
153,katsina,NG,12.9894,7.6006,28,Fadama view Hotel
226,cabo san lucas,MX,22.8909,-109.9124,46,Comfort Rooms
276,lompoc,US,34.6391,-120.4579,69,Embassy Suites by Hilton Lompoc Central Coast
360,saint-pierre,RE,-21.3393,55.4781,83,Tropic Hotel
473,kiama,AU,-34.6833,150.8667,64,Nova Kiama


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size="Humidity",
    color = "City",
    hover_cols=["City", "Humidity", "Hotel Name", "Country"],
    frame_width = 800,
    frame_height = 600    
)
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)